資料來源:歐洲某家銀行客戶的資料(https://www.kaggle.com/hemantc/churn-modelling/data)

背景敘述:我們建立一個類神經網路(深度學習)，來對我們的資料進行預測。我們針對資料中客戶的詳細資訊來判斷客戶是否會離開此家銀行。

Importing the libraries
導入標準庫

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Importing the dataset
導入資料集並且顯示我們的資料

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')

In [3]:
def load_csv_data(file):
    return pd.read_csv(file)
meau = load_csv_data("Churn_Modelling.csv")
meau.head()#只會顯示前五筆資料

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Feature如下:
RowNumber
CustomerId	    (客戶編號)
Surname	        (姓氏)
CreditScore	    (信用分數)
Geography	    (國家)
Gender	        (性別)
Age	            (年齡)
Tenure	        (客戶任期)
Balance	        (存款餘額)
NumOfProducts	(購買銀行幾項產品)
HasCrCard	    (是否擁有該家銀行信用卡)
IsActiveMember	(是否為活躍客戶)
EstimatedSalary	(估計薪水)
Exited	        (是否離開此家銀行)

Setting argument and independent variable
選擇我們的自變量以及因變量

我們選擇對因變量可能有影響的feature

自變量(X)選擇:信用分數、國家、性別、年齡、客戶任期、存款餘額、購買幾項銀行產品、是否擁有該家銀行信用卡、是否為活躍客戶、估計薪水

因變量(Y)選擇:是否離開此家銀行

In [4]:
X = dataset.iloc[:, 3:13].values #select all rows and 3rd-12th columns
y = dataset.iloc[:, 13].values #select all rows and 13th columns

Encoding categorical data
針對分類數據(國家、性別)進行處理

國家(X[:, 1]):法國對應0，德國對應1，西班牙對應2
因為國家並非只有0、1，所以我們使用OneHotEncoder對分類數據進行編碼

性別(X[:, 2]):女性對應到的值為0，男性對應到的值為1

避免虛擬變量陷阱，原本是三個變量，後來改為兩個

remainder='passthrough':all remaining columns that were not specified in transformers will be automatically passed through. ColumnTransformer其中的參數[1]:表示更改第一行的值

In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
ct=ColumnTransformer([('Country',OneHotEncoder(),[1])],remainder='passthrough')
X = ct.fit_transform(X)
X = X[:, 1:] #避免虛擬變量陷阱，原本國家是三個變量，後來改為兩個


Splitting the dataset into the Training set and Test set
將一萬個數據分成8000個訓練集跟2000個測試集

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

特徵縮放(Feature Scaling)
因為我們要避免一個自變量遠大於其他自變量

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Importing the Keras libraries and packages
導入Keras的標準庫以及工具包(Sequential,Dense)

如何安裝:打開終端機(prompt)，輸入指令安裝
conda install tensorflow
conda install -c conda-forge keras
conda update --all

Sequential:幫助我們初始化神經網路
Dense:初始化神經網路後要加上一層一層的網路(輸入層、隱藏層、輸出層)

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense

創建人工神經網路(輸入層、隱藏層、輸出層)
利用隨機梯度下降(Stochastic Gradient Descent)來訓練神經網路

Step1:隨機初始化權重，使得權重接近0但不為0。

Step2:將第一個觀察數據輸入"輸入層(input layer)"，每一個自變量特徵佔據一個神經元(input neurun)。

Step3:正向傳播:神經元由左至右傳入，活絡每一個神經元。每個神經元得到的值都與上一層神經元輸入的值有關。

Step4:比較神經網路輸出結果(預測值)和實際觀察的結果，計算其中的誤差。

Step5:反向傳播:將誤差由右至左反向傳播，依據誤差相對於權重的梯度，對每個權重進行更新，以達到最小化損失函數的目標。更新的速率是由學習速率以及梯度共同決定。

Step6:
針對一個新的數據進行觀察，並且重複Step1-Step5。(強化式學習reinforcement learning)
針對一組新的數據進行觀察，並且重複Step1-Step5。(批次學習batch learning)
此時損失函數的值是這一組所有觀察數據產生誤差的總和。

Step7:當整個訓練集都被輸入到神經網路以後，稱作一"期(Epoch)"訓練。我們可以進行更多"期(Epoch)"的訓練，優化我們的模型，提高我們的預測結果。

In [9]:
classifier = Sequential()

Adding the input layer and the first hidden layer
增加輸入層以及第一個隱藏層

Rectifier函數:適合用在隱藏層 Relu(X)=max(X,0)
Sigmoid函數:適合用在輸出層，因為函數輸出的值位在0跟1之間

unit:代表這一層輸出的維度有多少，這一層有多少神經元。
經過實驗經驗unit的選擇通常是(輸入層+輸出層)/2，舉例:(11+1)/2=6。

activation:代表函數選擇("relu"or"sigmoid")。

kernel_initializer:代表初始化權重的一個方法("uniform"代表隨機初始化)。

input_dim:輸入層的神經元個數。

In [10]:
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

加入第二層隱藏層

In [11]:
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

加入輸出層
unit:更改為1(輸出的神經元個數)，因為我們只要輸出1個結果。
activation:更改為"sigmoid"(只能判斷一個類別的機率為何)
(若要分類結果很多個，activation應該改為"softmax")

In [12]:
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

編譯神經網路
optimizer:決定step5最小化損失函數的方法

loss:損失函數，binary_crossentropy適用在分類結果為二進制，若要分為很多類則使用categorical_crossentropy

metrics:性能評估，其中accuracy為準確度

In [13]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

將訓練集放入模型進行訓練

In [14]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 1s 703us/step - loss: 0.4815 - accuracy: 0.7959
Epoch 2/100
800/800 [==============================] - 1s 734us/step - loss: 0.4269 - accuracy: 0.7960
Epoch 3/100
800/800 [==============================] - 1s 683us/step - loss: 0.4197 - accuracy: 0.8170
Epoch 4/100
800/800 [==============================] - 1s 687us/step - loss: 0.4134 - accuracy: 0.8296
Epoch 5/100
800/800 [==============================] - 1s 669us/step - loss: 0.4093 - accuracy: 0.8342
Epoch 6/100
800/800 [==============================] - 1s 692us/step - loss: 0.4065 - accuracy: 0.83530s - loss: 0.4030 - accuracy: 0.83
Epoch 7/100
800/800 [==============================] - 1s 659us/step - loss: 0.4042 - accuracy: 0.8351
Epoch 8/100
800/800 [==============================] - 1s 709us/step - loss: 0.4028 - accuracy: 0.8344
Epoch 9/100
800/800 [==============================] - 1s 651us/step - loss: 0.4015 - accuracy: 0.8344
Epoch 10/100
800/800 [=================

數據預測結果
若y_pred大於0.5則為true(表示會離開此家銀行)

In [15]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

以混淆矩陣呈現結果

In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [17]:
print('Confusion Matrix: \n', cm)

Confusion Matrix: 
 [[1509   86]
 [ 193  212]]
